### Runpod 설정

In [2]:
import os

# 모델 저장 경로를 네트워크 볼륨(/workspace) 내부로 지정
os.environ["HF_HOME"] = "/workspace/hf_cache"

In [ ]:
# 필수 라이브러리 설치/업데이트
!pip install transformers accelerate bitsandbytes openai torch

In [ ]:
# 필수 라이브러리 설치/업데이트
!pip install huggingface_hub

In [10]:
from huggingface_hub import notebook_login
notebook_login()

---

In [3]:
script = "상담사: 상담원 ▲▲▲입니다.\n손님: 네, 안녕하세요? 제가 오늘 오전에 전화해가지고 네 ▲▲카드 뭐 단기 요청을 드렸었는데\n상담사: 네, 단기 요청 말씀 하시시는 걸까요? 네.\n상담사: 네.\n손님: 네, 지금 보니까 제가 예 한 며칠 전에 쓴 게 오늘 결제가 된 거 같이서 그거 정보는 풀어 주셔도 될 것 같거든요.\n상담사: 네.\n상담사: 아 그러시군요. 네 혹시 해당 카드 지금 소지하고 계신가요?\n손님: 네.\n손님: 아 그게 지금 어 잠시만요. 뭐 거기 적혀있는 정보가 필요한 건가요?\n상담사: 네.\n상담사: 예, 저희 카드 인증이 필요하시기 때문에요, 저희 이제\n손님: 예.\n상담사: 한번 여쭤봤습니다, 네.\n손님: 그렇죠?\n손님: 아 네, 카드 가지고 있잖아요.\n상담사: 아 네. 저희 어 명의자님 성함은 ▲▲▲ 님 본인 맞으시고요?\n손님: 네.\n상담사: 예 찾으신 카드 재 정지 해제 원하시는 카드 끝에 번호 네 자리만 말씀 부탁드리겠습니다.\n손님: ▲▲▲▲▲▲▲\n상담사: 아 네, 말씀 감사드립니다. 혹시 저희가 그 등록된\n상담사: 어 핸드폰 번호로 문자를 보내드리면 인증 번호 확인 가능하신가요?\n손님: 어 어 한국 핸드폰 유심칩이 지금\n손님: 어 쓰기가 어렵거든요.\n상담사: 아 알겠습니다, 우선 그러시면\n손님: 아니면 카톡으로\n상담사: 아 그러셨군요.\n손님: 네.\n상담사: 네, 아니시면 제가 다른 인증 방법을 한번 여쭤볼 텐데요, 그 연결돼 있는 저희 ▲▲은행 계좌 번호 혹시 말씀 가능하실까요?\n손님: 네.\n손님: 아 네. \n손님: 어 네. ▲▲▲▲\n상담사: 네.\n손님: ▲▲▲▲▲▲▲\n상담사: 네\n손님: ▲▲▲▲▲▲▲\n상담사: 아 네, 확인되었습니다. 저희가 일단은 바로 정지 해지해드려서요. 어 ▲▲ 신용카드 지금부터 바로 이용 가능하십니다.\n손님: 네.\n손님: 아 네, 알겠습니다. 감사합니다.\n상담사: 네, 감사합니다. 상담원 ▲▲▲이었습니다.\n손님: 네."

system_prompt = f"""
        상답 스크립트를 분석하여 후처리 문서를 작성하세요.
        반드시 아래 JSON 형식을 지키세요.
        JSON 외의 서론, 결론, 마크다운 기호(```), 설명은 절대 포함하지 마세요.

        ### 상담 스크립트
        {script}

        ### 형식 (반드시 이 키값을 가진 JSON 하나만 출력)
        {{
            "상담 유형": "상담 유형",
            "문의 사항": "고객의 구체적인 문의 내용 1줄로 요약",
            "상담 과정": "단계별 상담 과정 요약",
            "처리 결과": "상담 결과 요약"
        }}
    """

### GPT 모델

In [ ]:
import time
import json
import openai

def generate_summary_openai(model_name, prompt):
    start_time = time.time()
    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[{"role": "system", "content": prompt}],
            temperature=0.1
        )

        content = response.choices[0].message.content

        result_json = json.loads(content)

        end_time = time.time()
        elapsed_time = end_time - start_time

        return result_json, round(elapsed_time, 2)
    except Exception as e:
        return f"{model_name} 호출 중 오류 발생 : {e}"

In [11]:
res, time = generate_summary_openai("gpt-4.1-mini", system_prompt)

In [13]:
res

{'상담 유형': '신용카드 단기 정지 해제 요청',
 '문의 사항': '고객이 단기 정지된 신용카드의 정지 해제를 요청함',
 '상담 과정': '고객이 단기 정지 요청을 했던 카드의 결제 정보 해제를 요청함 → 상담사가 카드 소지 여부 및 명의자 확인 → 카드 번호 끝 4자리 확인 → 인증 방법 문의 (휴대폰 문자 불가, 카톡 또는 계좌번호 인증 요청) → 고객이 은행 계좌번호 제공 → 상담사가 정보 확인 후 카드 정지 해제 처리 안내',
 '처리 결과': '고객의 신용카드 단기 정지 해제가 완료되어 즉시 이용 가능함'}

In [14]:
time

3.82

In [20]:
res_gpt4o, time_gpt4o = generate_summary_openai("gpt-4o", system_prompt)

In [21]:
res_gpt4o

{'상담 유형': '카드 정지 해제 요청',
 '문의 사항': '고객이 카드 결제 정보 확인 후 카드 정지 해제를 요청',
 '상담 과정': '고객이 카드 소지 여부 확인 후 카드 끝 4자리 번호 제공, 핸드폰 인증이 어려워 계좌 번호로 인증 진행',
 '처리 결과': '카드 정지 해제 완료, 고객이 카드 사용 가능'}

In [22]:
time_gpt4o

3.12

### sllm

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
import re
import json
import time

def generate_summary_sllm(model_name, prompt):
    start_time = time.time()
    try:
        # 모델 및 토크나이저 로드
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id
        
        messages = [{"role": "system", "content": prompt}]
        
        inputs = tokenizer.apply_chat_template(
            messages, 
            tokenize=True, 
            add_generation_prompt=True, 
            return_tensors="pt"
        ).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_new_tokens=512,
                repetition_penalty=1.2,
                do_sample=False,
                temperature=0.1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id
            )

        raw_content = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True).strip()
        
        # JSON만 추출
        try:
            clean_text = raw_content.split('assistant')[0].strip()
            
            # 정규표현식으로 가장 바깥쪽의 { } 구간만 추출
            json_match = re.search(r'(\{.*\})', clean_text, re.DOTALL)
        
            if json_match:
                json_str = json_match.group(1)
                json_str = json_str.replace('```json', '').replace('```', '').strip()
                
                # JSON 파싱
                content = json.loads(json_str)
            else:
                content = {"error": "JSON 형식을 찾을 수 없음", "raw_extracted": clean_text}
        except json.JSONDecodeError:
            content = {"error": "JSON 파싱 실패", "raw": raw_content}
        
        # 메모리 해제
        del model
        del tokenizer
        torch.cuda.empty_cache()

        end_time = time.time()
        elapsed_time = end_time - start_time
        return content, round(elapsed_time, 2)
        
    except Exception as e:
        return f"{model_name} 호출 중 오류 발생 : {e}"

In [ ]:
res_kanana_sm, time_kanana_sm = generate_summary_sllm("kakaocorp/kanana-nano-2.1b-instruct", system_prompt)

In [43]:
res_kanana_sm

{'상담 유형': '금융 서비스 관련 상담',
 '문의 사항': '단기 카드 사용 요청 취소 및 재개 요청',
 '상담 과정': '고객이 전날 발급받은 카드의 결제 내역 조회 요청 -> 고객이 카드 소지 여부 확인 -> 카드 인증 위해 연락처 또는 은행 계좌번호 입력 요구 -> 카드 정지 해제 처리 완료',
 '처리 결과': '고객의 단기 카드 사용 요청이 성공적으로 처리되어 카드 재개됨'}

In [ ]:
time_kanana_sm

In [ ]:
res_kanana_lg, time_kanana_lg = generate_summary_sllm("kakaocorp/kanana-1.5-8b-instruct-2505", system_prompt)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [45]:
res_kanana_lg

{'상담 유형': '카드 단기(정지/해제) 관련 문의',
 '문의 사항': '오전 단기 요청한 카드의 결제 및 즉시 사용 가능 상태 확인 및 처리 요청',
 '상담 과정': '상담사는 고객의 카드 단기 요청 내역을 확인하고, 현재 결제 완료 사실과 카드 사용 가능 상태를 안내함. 이후 카드 인증을 위해 고객에게 카드번호와 은행 계좌번호 등 추가 정보를 요구했고, 고객 동의 하에 카드 정지 해제 조치 및 즉시 사용 가능임을 고지함.',
 '처리 결과': '고객의 요청에 따라 해당 카드를 즉시 정지 해제하였으며, 카드는 현재 사용 가능한 상태임을 최종적으로 안내함.'}

In [ ]:
time_kanana_lg

In [ ]:
res_clovax, time_clovax = generate_summary_sllm("naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B", system_prompt)

In [46]:
res_clovax

{'error': 'JSON 파싱 실패',
 'raw': '```json\n{\n    "상담 유형": "도난 / 분실 신청 / 해제",\n    "문의 사항": "단기 요청한 카드의 정지 해제 요청을 하고 있습니다.",\n    "상담 과정": "고객이 단기 요청한 카드 정지 해제를 요청하며, 카드 정보와 본인 확인을 위해 명의자 성함과 카드 번호를 제공했습니다. 이후, 고객은 본인 확인을 위해 핸드폰 번호나 계좌번호를 인증할 수 있는지 문의하였고, 불가능하여 다른 인증 방법을 제안받았습니다.",\n    "처리 결과": "고객은 카드 정지 해제를 요청하였고, 상담사는 고객의 요청에 따라 카드를 정지 해제한 후, 즉시 카드 이용이 가능하도록 조치했습니다."\n}\n````````````````````````````````````````````````\nassistant\n```json\n{\n    "상담 유형": "도난 / 분실 신청 / 해제",\n    "문의 사항": "단기 요청한 카드의 정지 해제를 요청하고 있습니다.",\n    "상담 과정": "고객이 단기 요청한 카드 정지 해제를 요청하며, 카드 정보와 본인 확인을 위해 명의자 성함과 카드 번호를 제공했습니다. 이후, 고객은 본인 확인을 위해 핸드폰 번호나 계좌번호를 인증할 수 있는지 문의하였고, 불가능하여 다른 인증 방법을 제안받았습니다.",\n    "처리 결과": "고객은 카드 정지 해제를 요청하였고, 상담사는 고객의 요청에 따라 카드를 정지 해제한 후, 즉시 카드 이용이 가능하도록 조치했습니다."\n}\n``````\nassistant\n```json\n{\n    "상담 유형": "도난 / 분실 신청 / 해제",\n    "문의 사항": "단기 요청한 카드의 정지 해제를 요청하고 있습니다.",\n    "상담 과정": "고객이 단기 요청한 카드 정지 해제를 요청하며, 카드 정보와 본인 확인을 위해 명의자 성함과 카드 번호를 제공했습니다. 이후, 고객은 본인 확인을 위해 핸드폰 번호나 계좌

In [ ]:
time_clovax